In [1]:
from mxnet import autograd, nd, init, gluon
from mxnet.gluon import data as gdata
from mxnet.gluon import utils as gutils
from mxnet.gluon import loss as gloss
from mxnet.gluon import nn, rnn
from mxnet.contrib import text
from time import time
import mxnet as mx
import d2lzh as d2l
import collections
import random
import tarfile
import math
import os
import io

# 10. ⾃然语⾔处理

## 10.6 求近义词和类比词
在“word2vec的实现”一节中，我们在小规模数据集上训练了一个word2vec词嵌入模型，并通过词向量的余弦相似度搜索近义词。实际中，在大规模语料上预训练的词向量常常可以应用到下游自然语言处理任务中。本节将演示如何用这些预训练的词向量来求近义词和类比词。我们还将在后面两节中继续应用预训练的词向量。

### 10.6.1 使用预训练的词向量
`MXNet`的`contrib.text`包提供了与自然语言处理相关的函数和类(更多参见`GluonNLP`工具包)。下面查看它目前提供的预训练词嵌入的名称。

In [2]:
text.embedding.get_pretrained_file_names().keys()

dict_keys(['glove', 'fasttext'])

给定词嵌入名称，可以查看该词嵌入提供了哪些预训练的模型。每个模型的词向量维度可能不同，或是在不同数据集上预训练得到的。

In [3]:
print(text.embedding.get_pretrained_file_names('glove'))

['glove.42B.300d.txt', 'glove.6B.50d.txt', 'glove.6B.100d.txt', 'glove.6B.200d.txt', 'glove.6B.300d.txt', 'glove.840B.300d.txt', 'glove.twitter.27B.25d.txt', 'glove.twitter.27B.50d.txt', 'glove.twitter.27B.100d.txt', 'glove.twitter.27B.200d.txt']


预训练的GloVe模型的命名规范大致是`模型.(数据集.)数据集词数.词向量维度.txt`。更多信息可以参考GloVe和fastText的项目网站：
+ GloVe项⽬[⽹站](https://nlp.stanford.edu/projects/glove/)
+ fastText项⽬[⽹站](https://fasttext.cc/)

下面我们使用基于维基百科子集预训练的50维GloVe词向量。第一次创建预训练词向量实例时会自动下载相应的词向量，因此需要联网。

In [4]:
glove_6b50d = text.embedding.create( 'glove', pretrained_file_name='glove.6B.50d.txt')

len(glove_6b50d)

400001

In [5]:
#通过词来获取它在词典中的索引，也可以通过索引获取词
glove_6b50d.token_to_idx['beautiful'], glove_6b50d.idx_to_token[3367]

(3367, 'beautiful')

### 10.6.2 应用预训练词向量
我们以GloVe模型为例展示预训练词向量的应用。

##### 求近义词
这里重新实现“word2vec的实现”一节中介绍过的使用余弦相似度来搜索近义词的算法。为了在求类比词时重用其中的求`kNN`的逻辑，我们将这部分逻辑单独封装在`knn`函数中。

In [6]:
def knn(W, x, k):
    # 添加的1e-9是为了数值稳定性 
    cos = nd.dot(W, x.reshape((-1,))) / ( (nd.sum(W * W, axis=1) + 1e-9).sqrt() * nd.sum(x * x).sqrt()) 
    topk = nd.topk(cos, k=k, ret_typ='indices').asnumpy().astype('int32') 
    return topk, [cos[i].asscalar() for i in topk]

然后，我们通过预训练词向量实例`embed`来搜索近义词。

In [7]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed.get_vecs_by_tokens([query_token]), k+1) 
    for i, c in zip(topk[1:], cos[1:]): # 除去输⼊词 
        print('cosine sim=%.3f: %s' % (c, (embed.idx_to_token[i])))

已创建的预训练词向量实例`glove_6b50d`的词典中含40万个词和1个特殊的未知词。除去输⼊词和未知词，我们从中搜索与`chip`语义最相近的3个词。

In [8]:
get_similar_tokens('chip', 3, glove_6b50d)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics


接下来查找`baby`和`beautiful`的近义词。

In [9]:
get_similar_tokens('baby', 3, glove_6b50d)

cosine sim=0.839: babies
cosine sim=0.800: boy
cosine sim=0.792: girl


In [10]:
get_similar_tokens('beautiful', 3, glove_6b50d)

cosine sim=0.921: lovely
cosine sim=0.893: gorgeous
cosine sim=0.830: wonderful


##### 求类比词
除了求近义词以外，我们还可以使用预训练词向量求词与词之间的类比关系。例如，`man`: `woman`:: `son` : `daughter`是一个类比例子：`man`之于`woman`相当于`son`之于`daughter`。

求类比词问题可以定义为：对于类比关系中的4个词$a : b :: c : d$，给定前3个词$a$、$b$和$c$，求$d$。设词$w$的词向量为$\text{vec}(w)$。求类比词的思路是，搜索与$\text{vec}(c)+\text{vec}(b)-\text{vec}(a)$的结果向量最相似的词向量。

In [11]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed.get_vecs_by_tokens([token_a, token_b, token_c])
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1) 
    return embed.idx_to_token[topk[0]]

get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

`首都-国家`类比：`beijing`之于`china`相当于`tokyo`之于什么？答案应该是`japan`。

In [12]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

`形容词-形容词最高级`类比：`bad`之于`worst`相当于`big`之于什么？答案应该是`biggest`。

In [13]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

`动词一般时-动词过去时`类比：`do`之于`did`相当于`go`之于什么？答案应该是`went`。

In [14]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'

## 10.7 ⽂本情感分类:使⽤循环神经⽹络
文本分类是自然语言处理的一个常见任务，它把一段不定长的文本序列变换为文本的类别。本节关注它的一个子问题：使用文本情感分类来分析文本作者的情绪。这个问题也叫`情感分析`(sentiment analysis），并有着广泛的应用。例如，我们可以分析用户对产品的评论并统计用户的满意度，或者分析用户对市场行情的情绪并用以预测接下来的行情。

同求近义词和类比词一样，文本分类也属于词嵌入的下游应用。在本节中，我们将应用预训练的词向量和含多个隐藏层的双向循环神经网络，来判断一段不定长的文本序列中包含的是正面还是负面的情绪。

### 10.7.1 文本情感分类数据集
我们使用斯坦福的IMDb数据集作为文本情感分类的数据集。这个数据集分为训练和测试用的两个数据集，分别包含25,000条从IMDb下载的关于电影的评论。在每个数据集中，标签为`正面`和`负面`的评论数量相等。

##### 读取数据集
首先下载数据集。

In [15]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def download_imdb(data_dir='./data'):
    url = ('http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz') 
    sha1 = '01ada507287d82875905620988597833ad4e0903' 
    fname = gutils.download(url, data_dir, sha1_hash=sha1) 
    with tarfile.open(fname, 'r') as f:
        f.extractall(data_dir)

download_imdb()

接下来，读取训练数据集和测试数据集。每个样本是一条评论及其对应的标签：1表示`正面`，0表示`负面`。

In [16]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def read_imdb(folder='train'):
    data = [] 
    for label in ['pos', 'neg']:
        folder_name = os.path.join('./data/aclImdb/', folder, label)
        for file in os.listdir(folder_name):
            with open(os.path.join(folder_name, file), 'rb') as f:
                review = f.read().decode('utf-8').replace('\n', '').lower() 
                data.append([review, 1 if label == 'pos' else 0]) 
    random.shuffle(data) 
    return data

train_data, test_data = read_imdb('train'), read_imdb('test')

##### 预处理数据集
我们需要对每条评论做分词，从而得到分好词的评论。这里定义的`get_tokenized_imdb`函数使用最简单的方法：基于空格进行分词。

In [17]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def get_tokenized_imdb(data):
    def tokenizer(text):
        return [tok.lower() for tok in text.split(' ')] 
    return [tokenizer(review) for review, _ in data]

现在，我们可以根据分好词的训练数据集来创建词典了。我们在这⾥过滤掉了出现次数少于5的词。

In [18]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def get_vocab_imdb(data): 
    tokenized_data = get_tokenized_imdb(data) 
    counter = collections.Counter([tk for st in tokenized_data for tk in st]) 
    return text.vocab.Vocabulary(counter, min_freq=5)

vocab = get_vocab_imdb(train_data) 
'# words in vocab:', len(vocab)

('# words in vocab:', 46151)

因为每条评论长度不一致所以不能直接组合成小批量，我们定义`preprocess_imdb`函数对每条评论进行分词，并通过词典转换成词索引，然后通过截断或者补`<pad>`(padding)符号来将每条评论长度固定成500。

In [19]:
# 本函数已保存在d2lzh包中⽅便以后使⽤
def preprocess_imdb(data, vocab):  
    max_l = 500 # 将每条评论通过截断或者补0，使得⻓度变成500

    def pad(x):
        return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))

    tokenized_data = get_tokenized_imdb(data) 
    features = nd.array([pad(vocab.to_indices(x)) for x in tokenized_data]) 
    labels = nd.array([score for _, score in data]) 
    return features, labels

##### 创建数据迭代器
现在，我们创建数据迭代器。每次迭代将返回⼀个小批量的数据。

In [20]:
batch_size = 64

train_set = gdata.ArrayDataset(*preprocess_imdb(train_data, vocab))
test_set = gdata.ArrayDataset(*preprocess_imdb(test_data, vocab)) 
train_iter = gdata.DataLoader(train_set, batch_size, shuffle=True) 
test_iter = gdata.DataLoader(test_set, batch_size)

打印第⼀个小批量数据的形状以及训练集中小批量的个数。

In [21]:
for X, y in train_iter:
    print('X', X.shape, 'y', y.shape) 
    break
'#batches:', len(train_iter)

X (64, 500) y (64,)


('#batches:', 391)

### 10.7.2 使用循环神经网络的模型
在这个模型中，每个词先通过嵌入层得到特征向量。然后，我们使用双向循环神经网络对特征序列进一步编码得到序列信息。最后，我们将编码的序列信息通过全连接层变换为输出。具体来说，我们可以将双向长短期记忆在最初时间步和最终时间步的隐藏状态连结，作为特征序列的表征传递给输出层分类。在下面实现的`BiRNN`类中，`Embedding`实例即嵌入层，`LSTM`实例即为序列编码的隐藏层，`Dense`实例即生成分类结果的输出层。

In [22]:
class BiRNN(nn.Block):
    def __init__(self, vocab, embed_size, num_hiddens, num_layers, **kwargs):
        super(BiRNN, self).__init__(**kwargs) 
        self.embedding = nn.Embedding(len(vocab), embed_size) 
        # bidirectional设为True即得到双向循环神经⽹络 
        self.encoder = rnn.LSTM(num_hiddens, num_layers=num_layers, bidirectional=True, input_size=embed_size) 
        self.decoder = nn.Dense(2)

    def forward(self, inputs):
        # inputs的形状是(批量⼤⼩, 词数)，因为LSTM需要将序列作为第⼀维，所以将输⼊转置后 
        # 再提取词特征，输出形状为(词数, 批量⼤⼩, 词向量维度) 
        embeddings = self.embedding(inputs.T) 
        # rnn.LSTM只传⼊输⼊embeddings，因此只返回最后⼀层的隐藏层在各时间步的隐藏状态
        # outputs形状是(词数, 批量⼤⼩, 2 * 隐藏单元个数) 
        outputs = self.encoder(embeddings) 
        # 连结初始时间步和最终时间步的隐藏状态作为全连接层输⼊。它的形状为 
        # (批量⼤⼩, 4 * 隐藏单元个数)
        encoding = nd.concat(outputs[0], outputs[-1])
        outs = self.decoder(encoding) 
        return outs

# 创建⼀个含两个隐藏层的双向循环神经⽹络
# embed_size, num_hiddens, num_layers, ctx = 100, 100, 2, d2l.try_all_gpus()
embed_size, num_hiddens, num_layers, ctx = 100, 100, 2, mx.cpu()
net = BiRNN(vocab, embed_size, num_hiddens, num_layers) 
net.initialize(init.Xavier(), ctx=ctx)

##### 加载预训练的词向量
由于情感分类的训练数据集并不是很大，为应对过拟合，我们将直接使用在更大规模语料上预训练的词向量作为每个词的特征向量。这里，我们为词典`vocab`中的每个词加载100维的`GloVe`词向量。

In [23]:
glove_embedding = text.embedding.create( 'glove', pretrained_file_name='glove.6B.100d.txt', vocabulary=vocab)

我们将⽤这些词向量作为评论中每个词的特征向量。注意，预训练词向量的维度需要与创建的模型中的嵌⼊层输出⼤小`embed_size`⼀致。此外，在训练中我们不再更新这些词向量。

In [24]:
net.embedding.weight.set_data(glove_embedding.idx_to_vec) 
net.embedding.collect_params().setattr('grad_req', 'null')

##### 训练模型
这时候就可以开始训练模型了。

```python
lr, num_epochs = 0.01, 5
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr}) 
loss = gloss.SoftmaxCrossEntropyLoss() 
d2l.train(train_iter, test_iter, net, loss, trainer, ctx, num_epochs)
```

最后，定义预测函数。

```python
def predict_sentiment(net, vocab, sentence):
    sentence = nd.array(vocab.to_indices(sentence), ctx=d2l.try_gpu()) 
    label = nd.argmax(net(sentence.reshape((1, -1))), axis=1) 
    return 'positive' if label.asscalar() == 1 else 'negative'

# 情感分类
predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'great']), predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'bad'])
```

## 10.8 文本情感分类:使用卷积神经网络(textCNN)
在“卷积神经网络”一章中我们探究了如何使用二维卷积神经网络来处理二维图像数据。在之前的语言模型和文本分类任务中，我们将文本数据看作只有一个维度的时间序列，并很自然地使用循环神经网络来表征这样的数据。其实，我们也可以将文本当作一维图像，从而可以用一维卷积神经网络来捕捉临近词之间的关联。本节将介绍将卷积神经网络应用到文本分析的开创性工作之一：`textCNN`。

### 10.8.1 一维卷积层
在介绍模型前我们先来解释一维卷积层的工作原理。与二维卷积层一样，一维卷积层使用一维的互相关运算。在一维互相关运算中，卷积窗口从输入数组的最左方开始，按从左往右的顺序，依次在输入数组上滑动。当卷积窗口滑动到某一位置时，窗口中的输入子数组与核数组按元素相乘并求和，得到输出数组中相应位置的元素。如`图10.4`所示，输入是一个宽为7的一维数组，核数组的宽为2。可以看到输出的宽度为$7-2+1=6$，且第一个元素是由输入的最左边的宽为2的子数组与核数组按元素相乘后再相加得到的：$0\times1+1\times2=2$。

<img src="images/10_04.png" style="width:600px;"/>

下面我们将一维互相关运算实现在`corr1d`函数里。它接受输入数组`X`和核数组`K`，并输出数组`Y`。

In [25]:
def corr1d(X, K): 
    w = K.shape[0]
    Y = nd.zeros((X.shape[0] - w + 1))
    for i in range(Y.shape[0]):
        Y[i] = (X[i: i + w] * K).sum()
    return Y

# 复现图10.4中一维互相关运算的结果
X, K = nd.array([0, 1, 2, 3, 4, 5, 6]), nd.array([1, 2])
corr1d(X, K)


[ 2.  5.  8. 11. 14. 17.]
<NDArray 6 @cpu(0)>

多输入通道的一维互相关运算也与多输入通道的二维互相关运算类似：在每个通道上，将核与相应的输入做一维互相关运算，并将通道之间的结果相加得到输出结果。`图10.5`展示了含3个输入通道的一维互相关运算，其中阴影部分为第一个输出元素及其计算所使用的输入和核数组元素：$0\times1+1\times2+1\times3+2\times4+2\times(-1)+3\times(-3)=2$。

<img src="images/10_05.png" style="width:600px;"/>

In [26]:
def corr1d_multi_in(X, K):
    # ⾸先沿着X和K的第0维(通道维)遍历。然后使⽤*将结果列表变成add_n函数的位置参数 
    #（positional argument）来进⾏相加 
    return nd.add_n(*[corr1d(x, k) for x, k in zip(X, K)])

X = nd.array(
    [[0, 1, 2, 3, 4, 5, 6],
     [1, 2, 3, 4, 5, 6, 7],
     [2, 3, 4, 5, 6, 7, 8]])

K = nd.array([[1, 2], [3, 4], [-1, -3]])
corr1d_multi_in(X, K)


[ 2.  8. 14. 20. 26. 32.]
<NDArray 6 @cpu(0)>

由二维互相关运算的定义可知，多输入通道的一维互相关运算可以看作单输入通道的二维互相关运算。如`图10.6`所示，我们也可以将`图10.5`中多输入通道的一维互相关运算以等价的单输入通道的二维互相关运算呈现。这里核的高等于输入的高。`图10.6`中的阴影部分为第一个输出元素及其计算所使用的输入和核数组元素：$2\times(-1)+3\times(-3)+1\times3+2\times4+0\times1+1\times2=2$。

<img src="images/10_06.png" style="width:600px;"/>

`图10.4`和`图10.5`中的输出都只有一个通道。我们在“多输入通道和多输出通道”一节中介绍了如何在二维卷积层中指定多个输出通道。类似地，我们也可以在一维卷积层指定多个输出通道，从而拓展卷积层中的模型参数。

### 10.8.2 时序最⼤池化层
类似地，我们有一维池化层。`textCNN`中使用的`时序最大池化`(max-over-time pooling)层实际上对应一维全局最大池化层：假设输入包含多个通道，各通道由不同时间步上的数值组成，各通道的输出即该通道所有时间步中最大的数值。因此，时序最大池化层的输入在各个通道上的时间步数可以不同。

为提升计算性能，我们常常将不同长度的时序样本组成一个小批量，并通过在较短序列后附加特殊字符(如0)令批量中各时序样本长度相同。这些人为添加的特殊字符当然是无意义的。由于时序最大池化的主要目的是抓取时序中最重要的特征，它通常能使模型不受人为添加字符的影响。

### 10.8.3 读取和预处理IMDb数据集
我们依然使⽤和上⼀节中相同的IMDb数据集做情感分析。

In [27]:
batch_size = 64
download_imdb()

train_data, test_data = read_imdb('train'), read_imdb('test')
vocab = get_vocab_imdb(train_data)
train_iter = gdata.DataLoader(gdata.ArrayDataset(*preprocess_imdb(train_data, vocab)), batch_size, shuffle=True)
test_iter = gdata.DataLoader(gdata.ArrayDataset(*preprocess_imdb(test_data, vocab)), batch_size)

### 10.8.4 textCNN模型
textCNN模型主要使用了一维卷积层和时序最大池化层。假设输入的文本序列由$n$个词组成，每个词用$d$维的词向量表示。那么输入样本的宽为$n$，高为1，输入通道数为$d$。textCNN的计算主要分为以下几步。
1. 定义多个一维卷积核，并使用这些卷积核对输入分别做卷积计算。宽度不同的卷积核可能会捕捉到不同个数的相邻词的相关性
2. 对输出的所有通道分别做时序最大池化，再将这些通道的池化输出值连结为向量
3. 通过全连接层将连结后的向量变换为有关各类别的输出。这一步可以使用丢弃层应对过拟合

`图10.7`用一个例子解释了textCNN的设计。这里的输入是一个有11个词的句子，每个词用6维词向量表示。因此输入序列的宽为11，输入通道数为6。给定2个一维卷积核，核宽分别为2和4，输出通道数分别设为4和5。因此，一维卷积计算后，4个输出通道的宽为$11-2+1=10$，而其他5个通道的宽为$11-4+1=8$。尽管每个通道的宽不同，我们依然可以对各个通道做时序最大池化，并将9个通道的池化输出连结成一个9维向量。最终，使用全连接将9维向量变换为2维输出，即正面情感和负面情感的预测。

<img src="images/10_07.png" style="width:600px;"/>

下面我们来实现textCNN模型。与上一节相比，除了用一维卷积层替换循环神经网络外，这里我们还使用了两个嵌入层，一个的权重固定，另一个的权重则参与训练。

In [28]:
class TextCNN(nn.Block):
    def __init__(self, vocab, embed_size, kernel_sizes, num_channels, **kwargs):
        super(TextCNN, self).__init__(**kwargs) 
        self.embedding = nn.Embedding(len(vocab), embed_size) 
        # 不参与训练的嵌⼊层 
        self.constant_embedding = nn.Embedding(len(vocab), embed_size) 
        self.dropout = nn.Dropout(0.5) 
        self.decoder = nn.Dense(2) 
        # 时序最⼤池化层没有权重，所以可以共⽤⼀个实例 
        self.pool = nn.GlobalMaxPool1D()
        # 创建多个⼀维卷积层 
        self.convs = nn.Sequential() 
        for c, k in zip(num_channels, kernel_sizes):
            self.convs.add(nn.Conv1D(c, k, activation='relu'))

    def forward(self, inputs):
        # 将两个形状是(批量⼤⼩, 词数, 词向量维度)的嵌⼊层的输出按词向量连结
        embeddings = nd.concat(self.embedding(inputs), self.constant_embedding(inputs), dim=2) 
        # 根据Conv1D要求的输⼊格式，将词向量维，即⼀维卷积层的通道维，变换到前⼀维 
        embeddings = embeddings.transpose((0, 2, 1)) 
        # 对于每个⼀维卷积层，在时序最⼤池化后会得到⼀个形状为(批量⼤⼩, 通道⼤⼩, 1)的 
        # NDArray。使⽤flatten函数去掉最后⼀维，然后在通道维上连结 
        encoding = nd.concat(*[nd.flatten(self.pool(conv(embeddings))) for conv in self.convs], dim=1) 
        # 应⽤丢弃法后使⽤全连接层得到输出 
        outputs = self.decoder(self.dropout(encoding)) 
        return outputs

创建⼀个TextCNN实例。它有3个卷积层，它们的核宽分别为3、4和5，输出通道数均为100。

In [29]:
embed_size, kernel_sizes, nums_channels = 100, [3, 4, 5], [100, 100, 100] 
# ctx = d2l.try_all_gpus() 
ctx = mx.cpu() 
net = TextCNN(vocab, embed_size, kernel_sizes, nums_channels) 
net.initialize(init.Xavier(), ctx=ctx)

##### 加载预训练的词向量
同上一节一样，加载预训练的100维`GloVe`词向量，并分别初始化嵌入层`embedding`和`constant_embedding`，前者权重参与训练，而后者权重固定。

In [30]:
glove_embedding = text.embedding.create('glove', pretrained_file_name='glove.6B.100d.txt', vocabulary=vocab) 
net.embedding.weight.set_data(glove_embedding.idx_to_vec) 
net.constant_embedding.weight.set_data(glove_embedding.idx_to_vec) 
net.constant_embedding.collect_params().setattr('grad_req', 'null')

##### 训练并评价模型
现在就可以训练模型了。

```python
lr, num_epochs = 0.001, 5
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr})
loss = gloss.SoftmaxCrossEntropyLoss()
d2l.train(train_iter, test_iter, net, loss, trainer, ctx, num_epochs)
```

下⾯，我们使⽤训练好的模型对两个简单句⼦的情感进⾏分类。

```python
d2l.predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'great']), d2l.predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'bad'])
```

## 10.9 编码器/解码器(seq2seq)
我们已经在前两节中表征并变换了不定长的输入序列。但在自然语言处理的很多应用中，输入和输出都可以是不定长序列。以机器翻译为例，输入可以是一段不定长的英语文本序列，输出可以是一段不定长的法语文本序列，例如：
+ 英语输入：`They`、`are`、`watching`、`.`
+ 法语输出：`Ils`、`regardent`、`.`

当输入和输出都是不定长序列时，我们可以使用`编码器/解码器`(encoder-decoder)或者`seq2seq模型`。这两个模型本质上都用到了两个循环神经网络，分别叫做编码器和解码器。编码器用来分析输入序列，解码器用来生成输出序列。

`图10.8`描述了使用编码器/解码器将上述英语句子翻译成法语句子的一种方法。在训练数据集中，我们可以在每个句子后附上特殊符号`<eos>`(end of sequence)以表示序列的终止。编码器每个时间步的输入依次为英语句子中的单词、标点和特殊符号`<eos>`。

`图10.8`中使用了编码器在最终时间步的隐藏状态作为输入句子的表征或编码信息。解码器在各个时间步中使用输入句子的编码信息和上个时间步的输出以及隐藏状态作为输入。 我们希望解码器在各个时间步能正确依次输出翻译后的法语单词、标点和特殊符号`<eos>`。 需要注意的是，解码器在最初时间步的输入用到了一个表示序列开始的特殊符号`<bos>`(beginning of sequence)。

<img src="images/10_08.png" style="width:600px;"/>
    
接下来，我们分别介绍编码器和解码器的定义。

### 10.9.1 编码器
编码器的作用是把一个不定长的输入序列变换成一个定长的背景变量$\boldsymbol{c}$，并在该背景变量中编码输入序列信息。编码器可以使用循环神经网络。

让我们考虑批量大小为1的时序数据样本。假设输入序列是$x_1,\ldots,x_T$，例如$x_i$是输入句子中的第$i$个词。在时间步$t$，循环神经网络将输入$x_t$的特征向量$\boldsymbol{x}_t$和上个时间步的隐藏状态$\boldsymbol{h}_{t-1}$变换为当前时间步的隐藏状态$\boldsymbol{h}_t$。我们可以用函数$f$表达循环神经网络隐藏层的变换：
+ $\boldsymbol{h}_t = f(\boldsymbol{x}_t, \boldsymbol{h}_{t-1})$

接下来，编码器通过自定义函数$q$将各个时间步的隐藏状态变换为背景变量：
+ $\boldsymbol{c} = q(\boldsymbol{h}_1, \ldots, \boldsymbol{h}_T)$

例如，当选择$q(\boldsymbol{h}_1, \ldots, \boldsymbol{h}_T) = \boldsymbol{h}_T$时，背景变量是输入序列最终时间步的隐藏状态$\boldsymbol{h}_T$。

以上描述的编码器是一个单向的循环神经网络，每个时间步的隐藏状态只取决于该时间步及之前的输入子序列。我们也可以使用双向循环神经网络构造编码器。在这种情况下，编码器每个时间步的隐藏状态同时取决于该时间步之前和之后的子序列(包括当前时间步的输入)，并编码了整个序列的信息。

### 10.9.2 解码器
刚刚已经介绍，编码器输出的背景变量$\boldsymbol{c}$编码了整个输入序列$x_1, \ldots, x_T$的信息。给定训练样本中的输出序列$y_1, y_2, \ldots, y_{T'}$，对每个时间步$t'$(符号与输入序列或编码器的时间步$t$有区别)，解码器输出$y_{t'}$的条件概率将基于之前的输出序列$y_1,\ldots,y_{t'-1}$和背景变量$\boldsymbol{c}$，即$P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \boldsymbol{c})$。

为此，我们可以使用另一个循环神经网络作为解码器。 在输出序列的时间步$t^\prime$，解码器将上一时间步的输出$y_{t^\prime-1}$以及背景变量$\boldsymbol{c}$作为输入，并将它们与上一时间步的隐藏状态$\boldsymbol{s}_{t^\prime-1}$变换为当前时间步的隐藏状态$\boldsymbol{s}_{t^\prime}$。因此，我们可以用函数$g$表达解码器隐藏层的变换：
+ $\boldsymbol{s}_{t^\prime} = g(y_{t^\prime-1}, \boldsymbol{c}, \boldsymbol{s}_{t^\prime-1})$

有了解码器的隐藏状态后，我们可以使用自定义的输出层和softmax运算来计算$P(y_{t^\prime} \mid y_1, \ldots, y_{t^\prime-1}, \boldsymbol{c})$，例如，基于当前时间步的解码器隐藏状态 $\boldsymbol{s}_{t^\prime}$、上一时间步的输出$y_{t^\prime-1}$以及背景变量$\boldsymbol{c}$来计算当前时间步输出$y_{t^\prime}$的概率分布。
    
### 10.9.3 训练模型
根据最大似然估计，我们可以最大化输出序列基于输入序列的条件概率：
$$\begin{aligned} P(y_1, \ldots, y_{T'} \mid x_1, \ldots, x_T) &= \prod_{t'=1}^{T'} P(y_{t'} \mid y_1, \ldots, y_{t'-1}, x_1, \ldots, x_T) \\ &= \prod_{t'=1}^{T'} P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \boldsymbol{c}), \end{aligned}$$

并得到该输出序列的损失：
+ $-\log P(y_1, \ldots, y_{T'} \mid x_1, \ldots, x_T) = \displaystyle -\sum_{t'=1}^{T'} \log P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \boldsymbol{c})$

在模型训练中，所有输出序列损失的均值通常作为需要最小化的损失函数。在`图10.8`所描述的模型预测中，我们需要将解码器在上一个时间步的输出作为当前时间步的输入。与此不同，在训练中我们也可以将标签序列(训练集的真实输出序列)在上一个时间步的标签作为解码器在当前时间步的输入。这叫作强制教学(teacher forcing)。

## 10.10 束搜索
本节我们介绍如何使用编码器/解码器来预测不定长的序列。

上一节提到，在准备训练数据集时，我们通常会在样本的输入序列和输出序列后面分别附上一个特殊符号`<eos>`表示序列的终止。为了便于讨论，假设解码器的输出是一段文本序列。设输出文本词典$\mathcal{Y}$(包含特殊符号`<eos>`)的大小为$\left|\mathcal{Y}\right|$，输出序列的最大长度为$T'$。所有可能的输出序列一共有$\mathcal{O}(\left|\mathcal{Y}\right|^{T'})$种。这些输出序列中所有特殊符号`<eos>`后面的子序列将被舍弃。

### 10.10.1 贪婪搜索
让我们先来看一个简单的解决方案：贪婪搜索(greedy search)。对于输出序列任一时间步$t'$，我们从$|\mathcal{Y}|$个词中搜索出条件概率最大的词
+ $y_{t'} = \displaystyle \operatorname*{argmax}_{y \in \mathcal{Y}} P(y \mid y_1, \ldots, y_{t'-1}, \boldsymbol{c})$

作为输出。一旦搜索出`<eos>`符号，或者输出序列长度已经达到了最大长度$T'$，便完成输出。

我们在描述解码器时提到，基于输入序列生成输出序列的条件概率是$\prod_{t'=1}^{T'} P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \boldsymbol{c})$。我们将该条件概率最大的输出序列称为最优输出序列。而贪婪搜索的主要问题是不能保证得到最优输出序列。

下面来看一个例子。假设输出词典里面有`A`、`B`、`C`和`<eos>`这4个词。`图10.9`中每个时间步下的4个数字分别代表了该时间步生成`A`、`B`、`C`和`<eos>`这4个词的条件概率。在每个时间步，贪婪搜索选取条件概率最大的词。因此，`图10.9`中将生成输出序列`A`、`B`、`C`和`<eos>`。该输出序列的条件概率是$0.5\times0.4\times0.4\times0.6 = 0.048$。

<img src="images/10_09.png" style="width:400px;"/>
    
接下来，观察`图10.10`演示的例子。与`图10.9`中不同，`图10.10`在时间步2中选取了条件概率第二大的词`C`。由于时间步3所基于的时间步1和2的输出子序列由图10.9中的`A`、`B`变为了`图10.10`中的`A`、`C`，`图10.10`中时间步3生成各个词的条件概率发生了变化。我们选取条件概率最大的词`B`。此时时间步4所基于的前3个时间步的输出子序列为`A`、`C`、`B`，与图10.9中的`A`、`B`、`C`不同。因此，`图10.10`中时间步4生成各个词的条件概率也与`图10.9`中的不同。我们发现，此时的输出序列`A`、`C`、`B`、`<eos>`的条件概率是$0.5\times0.3\times0.6\times0.6=0.054$，大于贪婪搜索得到的输出序列的条件概率。因此，贪婪搜索得到的输出序列`A`、`B`、`C`、`<eos>`并非最优输出序列。

<img src="images/10_10.png" style="width:350px;"/>

### 10.10.2 穷举搜索
如果目标是得到最优输出序列，我们可以考虑`穷举搜索`(exhaustive search)：穷举所有可能的输出序列，输出条件概率最大的序列。

虽然穷举搜索可以得到最优输出序列，但它的计算开销$\mathcal{O}(\left|\mathcal{Y}\right|^{T'})$很容易过大。例如，当$|\mathcal{Y}|=10000$且$T'=10$时，我们将评估$10000^{10} = 10^{40}$个序列：这几乎不可能完成。而贪婪搜索的计算开销是$\mathcal{O}(\left|\mathcal{Y}\right|T')$，通常显著小于穷举搜索的计算开销。例如，当$|\mathcal{Y}|=10000$且$T'=10$时，我们只需评估$10000\times10=10^5$个序列。

### 10.10.3 束搜索
`束搜索`(beam search)是对贪婪搜索的一个改进算法。它有一个`束宽`(beam size)超参数。我们将它设为$k$。在时间步1时，选取当前时间步条件概率最大的$k$个词，分别组成$k$个候选输出序列的首词。在之后的每个时间步，基于上个时间步的$k$个候选输出序列，从$k\left|\mathcal{Y}\right|$个可能的输出序列中选取条件概率最大的$k$个，作为该时间步的候选输出序列。最终，我们从各个时间步的候选输出序列中筛选出包含特殊符号`<eos>`的序列，并将它们中所有特殊符号`<eos>`后面的子序列舍弃，得到最终候选输出序列的集合。
    
<img src="images/10_11.png" style="width:600px;"/>
    
`图10.11`通过一个例子演示了束搜索的过程。假设输出序列的词典中只包含5个元素，即$\mathcal{Y} = {A, B, C, D, E}$，且其中一个为特殊符号`<eos>`。设束搜索的束宽等于2，输出序列最大长度为3。在输出序列的时间步1时，假设条件概率$P(y_1 \mid \boldsymbol{c})$最大的2个词为$A$和$C$。我们在时间步2时将对所有的$y_2 \in \mathcal{Y}$都分别计算$P(A, y_2 \mid \boldsymbol{c}) = P(A \mid \boldsymbol{c})P(y_2 \mid A, \boldsymbol{c})$和$P(C, y_2 \mid \boldsymbol{c}) = P(C \mid \boldsymbol{c})P(y_2 \mid C, \boldsymbol{c})$，并从计算出的10个条件概率中取最大的2个，假设为$P(A, B \mid \boldsymbol{c})$和$P(C, E \mid \boldsymbol{c})$。那么，我们在时间步3时将对所有的$y_3 \in \mathcal{Y}$都分别计算$P(A, B, y_3 \mid \boldsymbol{c}) = P(A, B \mid \boldsymbol{c})P(y_3 \mid A, B, \boldsymbol{c})$和$P(C, E, y_3 \mid \boldsymbol{c}) = P(C, E \mid \boldsymbol{c})P(y_3 \mid C, E, \boldsymbol{c})$，并从计算出的10个条件概率中取最大的2个，假设为$P(A, B, D \mid \boldsymbol{c})$和$P(C, E, D \mid \boldsymbol{c})$。如此一来，我们得到6个候选输出序列：
1. $A$
2. $C$
3. $A$、$B$
4. $C$、$E$
5. $A$、$B$、$D$
6. $C$、$E$、$D$

接下来，我们将根据这6个序列得出最终候选输出序列的集合。

在最终候选输出序列的集合中，我们取以下分数最高的序列作为输出序列：
+ $\displaystyle \frac{1}{L^\alpha} \log P(y_1, \ldots, y_{L}) = \frac{1}{L^\alpha} \sum_{t'=1}^L \log P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \boldsymbol{c})$

其中$L$为最终候选序列长度，$\alpha$一般可选为0.75。分母上的$L^\alpha$是为了惩罚较长序列在以上分数中较多的对数相加项。分析可知，束搜索的计算开销为$\mathcal{O}(k\left|\mathcal{Y}\right|T')$。这介于贪婪搜索和穷举搜索的计算开销之间。此外，贪婪搜索可看作是束宽为1的束搜索。束搜索通过灵活的束宽$k$来权衡计算开销和搜索质量。

## 10.11 注意力机制
在“编码器/解码器(seq2seq)”一节里，解码器在各个时间步依赖相同的背景变量来获取输入序列信息。当编码器为循环神经网络时，背景变量来自它最终时间步的隐藏状态。

现在，让我们再次思考那一节提到的翻译例子：输入为英语序列`They`、`are`、`watching`、`.`，输出为法语序列`Ils`、`regardent`、`.`。不难想到，解码器在生成输出序列中的每一个词时可能只需利用输入序列某一部分的信息。例如，在输出序列的时间步1，解码器可以主要依赖`They`、`are`的信息来生成`Ils`，在时间步2则主要使用来自`watching`的编码信息生成`regardent`，最后在时间步3则直接映射句号`.`。这看上去就像是在解码器的每一时间步对输入序列中不同时间步的表征或编码信息分配不同的注意力一样。这也是注意力机制的由来。

仍然以循环神经网络为例，注意力机制通过对编码器所有时间步的隐藏状态做加权平均来得到背景变量。解码器在每一时间步调整这些权重，即注意力权重，从而能够在不同时间步分别关注输入序列中的不同部分并编码进相应时间步的背景变量。本节我们将讨论注意力机制是怎么工作的。

在“编码器/解码器(seq2seq)”一节里我们区分了输入序列或编码器的索引$t$与输出序列或解码器的索引$t'$。该节中，解码器在时间步$t'$的隐藏状态$\boldsymbol{s}_{t'} = g(\boldsymbol{y}_{t'-1}, \boldsymbol{c}, \boldsymbol{s}_{t'-1})$，其中$\boldsymbol{y}_{t'-1}$是上一时间步$t'-1$的输出$y_{t'-1}$的表征，且任一时间步$t'$使用相同的背景变量$\boldsymbol{c}$。但在注意力机制中，解码器的每一时间步将使用可变的背景变量。记$\boldsymbol{c}_{t'}$是解码器在时间步$t'$的背景变量，那么解码器在该时间步的隐藏状态可以改写为
+ $\boldsymbol{s}_{t'} = g(\boldsymbol{y}_{t'-1}, \boldsymbol{c}_{t'}, \boldsymbol{s}_{t'-1})$

这里的关键是如何计算背景变量$\boldsymbol{c}_{t'}$和如何利用它来更新隐藏状态$\boldsymbol{s}_{t'}$。下面将分别描述这两个关键点。

### 10.11.1 计算背景变量
我们先描述第一个关键点，即计算背景变量。`图10.12`描绘了注意力机制如何为解码器在时间步2计算背景变量。首先，函数$a$根据解码器在时间步1的隐藏状态和编码器在各个时间步的隐藏状态计算softmax运算的输入。softmax运算输出概率分布并对编码器各个时间步的隐藏状态做加权平均，从而得到背景变量。

<img src="images/10_12.png" style="width:500px;"/>

具体来说，令编码器在时间步$t$的隐藏状态为$\boldsymbol{h}_t$，且总时间步数为$T$。那么解码器在时间步$t'$的背景变量为所有编码器隐藏状态的加权平均：
+ $\boldsymbol{c}_{t'} = \displaystyle\sum_{t=1}^T \alpha_{t' t} \boldsymbol{h}_t$

其中给定$t'$时，权重$\alpha_{t' t}$在$t=1,\ldots,T$的值是一个概率分布。为了得到概率分布，我们可以使用softmax运算:
+ $\alpha_{t' t} = \displaystyle\frac{\exp(e_{t' t})}{\displaystyle \sum_{k=1}^T \exp(e_{t' k}) },\quad t=1,\ldots,T$

现在，我们需要定义如何计算上式中softmax运算的输入$e_{t' t}$。由于$e_{t' t}$同时取决于解码器的时间步$t'$和编码器的时间步$t$，我们不妨以解码器在时间步$t'-1$的隐藏状态$\boldsymbol{s}_{t' - 1}$与编码器在时间步$t$的隐藏状态$\boldsymbol{h}t$为输入，并通过函数$a$计算$e{t' t}$：
+ $e_{t' t} = a(\boldsymbol{s}_{t' - 1}, \boldsymbol{h}_t)$

这里函数$a$有多种选择，如果两个输入向量长度相同，一个简单的选择是计算它们的内积$a(\boldsymbol{s}, \boldsymbol{h})=\boldsymbol{s}^\top \boldsymbol{h}$。而最早提出注意力机制的论文则将输入连结后通过含单隐藏层的多层感知机变换 [1]：
+ $a(\boldsymbol{s}, \boldsymbol{h}) = \boldsymbol{v}^\top \tanh(\boldsymbol{W}_s \boldsymbol{s} + \boldsymbol{W}_h \boldsymbol{h})$

其中$\boldsymbol{v}$、$\boldsymbol{W}_s$、$\boldsymbol{W}_h$都是可以学习的模型参数。

##### 矢量化计算
我们还可以对注意力机制采用更高效的矢量化计算。广义上，注意力机制的输入包括查询项以及一一对应的键项和值项，其中值项是需要加权平均的一组项。在加权平均中，值项的权重来自查询项以及与该值项对应的键项的计算。

在上面的例子中，查询项为解码器的隐藏状态，键项和值项均为编码器的隐藏状态。 让我们考虑一个常见的简单情形，即编码器和解码器的隐藏单元个数均为$h$，且函数$a(\boldsymbol{s}, \boldsymbol{h})=\boldsymbol{s}^\top \boldsymbol{h}$。假设我们希望根据解码器单个隐藏状态$\boldsymbol{s}_{t' - 1} \in \mathbb{R}^{h}$和编码器所有隐藏状态$\boldsymbol{h}_t \in \mathbb{R}^{h}, t = 1,\ldots,T$来计算背景向量$\boldsymbol{c}_{t'}\in \mathbb{R}^{h}$。 我们可以将查询项矩阵$\boldsymbol{Q} \in \mathbb{R}^{1 \times h}$设为$\boldsymbol{s}_{t' - 1}^\top$，并令键项矩阵$\boldsymbol{K} \in \mathbb{R}^{T \times h}$和值项矩阵$\boldsymbol{V} \in \mathbb{R}^{T \times h}$相同且第$t$行均为$\boldsymbol{h}_t^\top$。此时，我们只需要通过矢量化计算：
+ $\mathcal{softmax}(\boldsymbol{Q}\boldsymbol{K}^\top)\boldsymbol{V}$

即可算出转置后的背景向量$\boldsymbol{c}_{t'}^\top$。当查询项矩阵$\boldsymbol{Q}$的行数为$n$时，上式将得到$n$行的输出矩阵。输出矩阵与查询项矩阵在相同行上一一对应。

### 10.11.2 更新隐藏状态
现在我们描述第二个关键点，即更新隐藏状态。以门控循环单元为例，在解码器中我们可以对“门控循环单元(GRU)”一节中门控循环单元的设计稍作修改，从而变换上一时间步$t'-1$的输出$\boldsymbol{y}_{t'-1}$、隐藏状态$\boldsymbol{s}_{t'-1}$和当前时间步$t'$的含注意力机制的背景变量$\boldsymbol{c}_{t'}$。

解码器在时间步$t'$的隐藏状态为：
$$\displaystyle\boldsymbol{s}_{t'} = \boldsymbol{z}_{t'} \odot \boldsymbol{s}_{t'-1} + (1 - \boldsymbol{z}_{t'}) \odot \tilde{\boldsymbol{s}}_{t'}$$

其中的重置门、更新门和候选隐藏状态分别为：
$$\begin{aligned} \boldsymbol{r}_{t'} &= \sigma(\boldsymbol{W}_{yr} \boldsymbol{y}_{t'-1} + \boldsymbol{W}_{sr} \boldsymbol{s}_{t' - 1} + \boldsymbol{W}_{cr} \boldsymbol{c}_{t'} + \boldsymbol{b}_r), \\ \boldsymbol{z}_{t'} &= \sigma(\boldsymbol{W}_{yz} \boldsymbol{y}_{t'-1} + \boldsymbol{W}_{sz} \boldsymbol{s}_{t' - 1} + \boldsymbol{W}_{cz} \boldsymbol{c}_{t'} + \boldsymbol{b}_z),\\ \tilde{\boldsymbol{s}}_{t'} &= \text{tanh}(\boldsymbol{W}_{ys} \boldsymbol{y}_{t'-1} + \boldsymbol{W}_{ss} (\boldsymbol{s}_{t' - 1} \odot \boldsymbol{r}_{t'}) + \boldsymbol{W}_{cs} \boldsymbol{c}_{t'} + \boldsymbol{b}_s) \end{aligned} $$

其中含下标的$\boldsymbol{W}$和$\boldsymbol{b}$分别为门控循环单元的权重参数和偏差参数。

### 10.11.3 发展
本质上，注意力机制能够为表征中较有价值的部分分配较多的计算资源。这个有趣的想法自提出后得到了快速发展，特别是启发了依靠注意力机制来编码输入序列并解码出输出序列的变换器(Transformer)模型的设计。变换器抛弃了卷积神经网络和循环神经网络的架构。它在计算效率上比基于循环神经网络的编码器/解码器模型通常更具明显优势。

含注意力机制的变换器的编码结构在后来的BERT预训练模型中得以应用并令后者大放异彩：微调后的模型在多达11项自然语言处理任务中取得了当时最先进的结果。不久后，同样是基于变换器设计的GPT-2模型于新收集的语料数据集预训练后，在7个未参与训练的语言模型数据集上均取得了当时最先进的结果。除了自然语言处理领域，注意力机制还被广泛用于图像分类、自动图像描述、唇语解读以及语音识别。

## 10.12 机器翻译
机器翻译是指将一段文本从一种语言自动翻译到另一种语言。因为一段文本序列在不同语言中的长度不一定相同，所以我们使用机器翻译为例来介绍编码器—解码器和注意力机制的应用。

### 10.12.1 读取和预处理数据集
我们先定义一些特殊符号。其中`<pad>`(padding)符号用来添加在较短序列后，直到每个序列等长，而`<bos>`和`<eos>`符号分别表示序列的开始和结束。

In [31]:
PAD, BOS, EOS = '<pad>', '<bos>', '<eos>'

两个辅助函数对后面读取的数据进行预处理。

In [32]:
# 将⼀个序列中所有的词记录在all_tokens中以便之后构造词典，然后在该序列后⾯添加PAD直到序列 
# ⻓度变为max_seq_len，然后将序列保存在all_seqs中
def process_one_seq(seq_tokens, all_tokens, all_seqs, max_seq_len):
    all_tokens.extend(seq_tokens) 
    seq_tokens += [EOS] + [PAD] * (max_seq_len - len(seq_tokens) - 1) 
    all_seqs.append(seq_tokens)

# 使⽤所有的词来构造词典。并将所有序列中的词变换为词索引后构造NDArray实例 
def build_data(all_tokens, all_seqs):
    vocab = text.vocab.Vocabulary(collections.Counter(all_tokens), reserved_tokens=[PAD, BOS, EOS])
    indices = [vocab.to_indices(seq) for seq in all_seqs] 
    return vocab, nd.array(indices)

为了演示方便，我们在这里使用一个很小的法语—英语数据集。在这个数据集里，每一行是一对法语句子和它对应的英语句子，中间使用`\t`隔开。在读取数据时，我们在句末附上`<eos>`符号，并可能通过添加`<pad>`符号使每个序列的长度均为`max_seq_len`。我们为法语词和英语词分别创建词典。法语词的索引和英语词的索引相互独立。

In [33]:
def read_data(max_seq_len):
    # in和out分别是input和output的缩写
    in_tokens, out_tokens, in_seqs, out_seqs = [], [], [], []
    with io.open('./data/fr-en-small.txt') as f:
        lines = f.readlines()
    for line in lines:
        in_seq, out_seq = line.rstrip().split('\t')
        in_seq_tokens, out_seq_tokens = in_seq.split(' '), out_seq.split(' ')
        if max(len(in_seq_tokens), len(out_seq_tokens)) > max_seq_len - 1:
            continue  # 如果加上EOS后长于max_seq_len，则忽略掉此样本
        process_one_seq(in_seq_tokens, in_tokens, in_seqs, max_seq_len)
        process_one_seq(out_seq_tokens, out_tokens, out_seqs, max_seq_len)
    in_vocab, in_data = build_data(in_tokens, in_seqs)
    out_vocab, out_data = build_data(out_tokens, out_seqs)
    return in_vocab, out_vocab, gdata.ArrayDataset(in_data, out_data)

将序列的最大长度设成7，然后查看读取到的第一个样本。该样本分别包含法语词索引序列和英语词索引序列。

In [34]:
max_seq_len = 7
in_vocab, out_vocab, dataset = read_data(max_seq_len)
dataset[0]

(
 [ 6.  5. 46.  4.  3.  1.  1.]
 <NDArray 7 @cpu(0)>,
 
 [ 9.  5. 28.  4.  3.  1.  1.]
 <NDArray 7 @cpu(0)>)

### 10.12.2 含注意力机制的编码器—解码器
我们将使用含注意力机制的编码器/解码器来将一段简短的法语翻译成英语。

##### 编码器
在编码器中，我们将输入语言的词索引通过词嵌入层得到词的表征，然后输入到一个多层门控循环单元中。正如我们在“循环神经网络的简洁实现”一节提到的，`Gluon`的`rnn.GRU`实例在前向计算后也会分别返回输出和最终时间步的多层隐藏状态。其中的输出指的是最后一层的隐藏层在各个时间步的隐藏状态，并不涉及输出层计算。注意力机制将这些输出作为键项和值项。

In [35]:
class Encoder(nn.Block):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, drop_prob=0, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = rnn.GRU(num_hiddens, num_layers, dropout=drop_prob)

    def forward(self, inputs, state):
        # 输入形状是(批量大小, 时间步数)。将输出互换样本维和时间步维
        embedding = self.embedding(inputs).swapaxes(0, 1)
        return self.rnn(embedding, state)

    def begin_state(self, *args, **kwargs):
        return self.rnn.begin_state(*args, **kwargs)

下面我们来创建一个批量大小为4、时间步数为7的小批量序列输入。设门控循环单元的隐藏层个数为2，隐藏单元个数为16。编码器对该输入执行前向计算后返回的输出形状为`(时间步数, 批量大小, 隐藏单元个数)`。门控循环单元在最终时间步的多层隐藏状态的形状为`(隐藏层个数, 批量大小, 隐藏单元个数)`。对于门控循环单元来说，`state`列表中只含一个元素，即隐藏状态；如果使用长短期记忆，state列表中还将包含另一个元素，即记忆细胞。

In [36]:
encoder = Encoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2) 
encoder.initialize() 
output, state = encoder(nd.zeros((4, 7)), encoder.begin_state(batch_size=4)) 
output.shape, state[0].shape

((7, 4, 16), (2, 4, 16))

##### 注意力机制
在介绍如何实现注意力机制的矢量化计算之前，我们先了解一下`Dense`实例的`flatten`选项。当输入的维度大于2时，默认情况下，`Dense`实例会将除了第一维(样本维)以外的维度均视作需要仿射变换的特征维，并将输入自动转成行为样本、列为特征的二维矩阵。计算后，输出矩阵的形状为`(样本数, 输出个数)`。如果我们希望全连接层只对输入的最后一维做仿射变换，而保持其他维度上的形状不变，便需要将`Dense`实例的`flatten`选项设为`False`。在下面例子中，全连接层只对输入的最后一维做仿射变换，因此输出形状中只有最后一维变为全连接层的输出个数2。

In [37]:
dense = nn.Dense(2, flatten=False) 
dense.initialize() 
dense(nd.zeros((3, 5, 7))).shape

(3, 5, 2)

我们将实现“注意力机制”一节中定义的函数$a$：将输入连结后通过含单隐藏层的多层感知机变换。其中隐藏层的输入是解码器的隐藏状态与编码器在所有时间步上隐藏状态的一一连结，且使用`tanh`函数作为激活函数。输出层的输出个数为1。两个`Dense`实例均不使用偏差，且设`flatten=False`。其中函数$a$定义里向量$\boldsymbol{v}$的长度是一个超参数，即`attention_size`。

In [38]:
def attention_model(attention_size):
    model = nn.Sequential()
    model.add(
        nn.Dense(attention_size, activation='tanh', use_bias=False, flatten=False),
        nn.Dense(1, use_bias=False, flatten=False))
    return model

注意力机制的输入包括查询项、键项和值项。设编码器和解码器的隐藏单元个数相同。这里的查询项为解码器在上一时间步的隐藏状态，形状为`(批量大小, 隐藏单元个数)`；键项和值项均为编码器在所有时间步的隐藏状态，形状为`(时间步数, 批量大小, 隐藏单元个数)`。注意力机制返回当前时间步的背景变量，形状为`(批量大小, 隐藏单元个数)`。

In [39]:
def attention_forward(model, enc_states, dec_state):
    # 将解码器隐藏状态⼴播到和编码器隐藏状态形状相同后进⾏连结 
    dec_states = nd.broadcast_axis( dec_state.expand_dims(0), axis=0, size=enc_states.shape[0]) 
    enc_and_dec_states = nd.concat(enc_states, dec_states, dim=2) 
    # 形状为(时间步数, 批量⼤⼩, 1)
    e = model(enc_and_dec_states) 
    # 在时间步维度做softmax运算 
    alpha = nd.softmax(e, axis=0) 
    # 返回背景变量
    return (alpha * enc_states).sum(axis=0) 

在下面的例子中，编码器的时间步数为10，批量大小为4，编码器和解码器的隐藏单元个数均为8。注意力机制返回一个小批量的背景向量，每个背景向量的长度等于编码器的隐藏单元个数。因此输出的形状为(4, 8)。

In [40]:
seq_len, batch_size, num_hiddens = 10, 4, 8

model = attention_model(10) 
model.initialize() 
enc_states = nd.zeros((seq_len, batch_size, num_hiddens)) 
dec_state = nd.zeros((batch_size, num_hiddens)) 
attention_forward(model, enc_states, dec_state).shape

(4, 8)

##### 含注意力机制的解码器
我们直接将编码器在最终时间步的隐藏状态作为解码器的初始隐藏状态。这要求编码器和解码器的循环神经网络使用相同的隐藏层个数和隐藏单元个数。

在解码器的前向计算中，我们先通过刚刚介绍的注意力机制计算得到当前时间步的背景向量。由于解码器的输入来自输出语言的词索引，我们将输入通过词嵌入层得到表征，然后和背景向量在特征维连结。我们将连结后的结果与上一时间步的隐藏状态通过门控循环单元计算出当前时间步的输出与隐藏状态。最后，我们将输出通过全连接层变换为有关各个输出词的预测，形状为(批量大小, 输出词典大小)。

In [41]:
class Decoder(nn.Block):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, attention_size, drop_prob=0, **kwargs):
        super(Decoder, self).__init__(**kwargs) 
        self.embedding = nn.Embedding(vocab_size, embed_size) 
        self.attention = attention_model(attention_size) 
        self.rnn = rnn.GRU(num_hiddens, num_layers, dropout=drop_prob) 
        self.out = nn.Dense(vocab_size, flatten=False)

    def forward(self, cur_input, state, enc_states):
        # 使⽤注意⼒机制计算背景向量 
        c = attention_forward(self.attention, enc_states, state[0][-1]) 
        # 将嵌⼊后的输⼊和背景向量在特征维连结 
        input_and_c = nd.concat(self.embedding(cur_input), c, dim=1) 
        # 为输⼊和背景向量的连结增加时间步维，时间步个数为1 
        output, state = self.rnn(input_and_c.expand_dims(0), state) 
        # 移除时间步维，输出形状为(批量⼤⼩, 输出词典⼤⼩) 
        output = self.out(output).squeeze(axis=0) 
        return output, state

    def begin_state(self, enc_state):
        # 直接将编码器最终时间步的隐藏状态作为解码器的初始隐藏状态 
        return enc_state

### 10.12.3 训练模型
我们先实现`batch_loss`函数计算一个小批量的损失。解码器在最初时间步的输入是特殊字符`BOS`。之后，解码器在某时间步的输入为样本输出序列在上一时间步的词，即强制教学。此外，同“word2vec的实现”一节中的实现一样，我们在这里也使用掩码变量避免填充项对损失函数计算的影响。

In [42]:
def batch_loss(encoder, decoder, X, Y, loss):
    batch_size = X.shape[0]
    enc_state = encoder.begin_state(batch_size=batch_size)
    enc_outputs, enc_state = encoder(X, enc_state)
    # 初始化解码器的隐藏状态
    dec_state = decoder.begin_state(enc_state)
    # 解码器在最初时间步的输入是BOS
    dec_input = nd.array([out_vocab.token_to_idx[BOS]] * batch_size)
    # 我们将使用掩码变量mask来忽略掉标签为填充项PAD的损失
    mask, num_not_pad_tokens = nd.ones(shape=(batch_size,)), 0
    l = nd.array([0])
    for y in Y.T:
        dec_output, dec_state = decoder(dec_input, dec_state, enc_outputs)
        l = l + (mask * loss(dec_output, y)).sum()
        dec_input = y  # 使用强制教学
        num_not_pad_tokens += mask.sum().asscalar()
        # 当遇到EOS时，序列后面的词将均为PAD，相应位置的掩码设成0
        mask = mask * (y != out_vocab.token_to_idx[EOS])
    return l / num_not_pad_tokens

在训练函数中，我们需要同时迭代编码器和解码器的模型参数。

In [43]:
def train(encoder, decoder, dataset, lr, batch_size, num_epochs):
    encoder.initialize(init.Xavier(), force_reinit=True)
    decoder.initialize(init.Xavier(), force_reinit=True)
    enc_trainer = gluon.Trainer(encoder.collect_params(), 'adam',
                                {'learning_rate': lr})
    dec_trainer = gluon.Trainer(decoder.collect_params(), 'adam',
                                {'learning_rate': lr})
    loss = gloss.SoftmaxCrossEntropyLoss()
    data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)
    for epoch in range(num_epochs):
        l_sum = 0.0
        for X, Y in data_iter:
            with autograd.record():
                l = batch_loss(encoder, decoder, X, Y, loss)
            l.backward()
            enc_trainer.step(1)
            dec_trainer.step(1)
            l_sum += l.asscalar()
        if (epoch + 1) % 10 == 0:
            print("epoch %d, loss %.3f" % (epoch + 1, l_sum / len(data_iter)))

接下来，创建模型实例并设置超参数。然后，我们就可以训练模型了。

In [44]:
embed_size, num_hiddens, num_layers = 64, 64, 2
attention_size, drop_prob, lr, batch_size, num_epochs = 10, 0.5, 0.01, 2, 50 
encoder = Encoder(len(in_vocab), embed_size, num_hiddens, num_layers, drop_prob) 
decoder = Decoder(len(out_vocab), embed_size, num_hiddens, num_layers, attention_size, drop_prob) 
train(encoder, decoder, dataset, lr, batch_size, num_epochs)

epoch 10, loss 0.613
epoch 20, loss 0.177
epoch 30, loss 0.137
epoch 40, loss 0.068
epoch 50, loss 0.050


### 10.12.4 预测不定长的序列
在“束搜索”一节中我们介绍了3种方法来生成解码器在每个时间步的输出。这里我们实现最简单的贪婪搜索。

In [45]:
def translate(encoder, decoder, input_seq, max_seq_len):
    in_tokens = input_seq.split(' ')
    in_tokens += [EOS] + [PAD] * (max_seq_len - len(in_tokens) - 1)
    enc_input = nd.array([in_vocab.to_indices(in_tokens)])
    enc_state = encoder.begin_state(batch_size=1)
    enc_output, enc_state = encoder(enc_input, enc_state)
    dec_input = nd.array([out_vocab.token_to_idx[BOS]])
    dec_state = decoder.begin_state(enc_state)
    output_tokens = []
    for _ in range(max_seq_len):
        dec_output, dec_state = decoder(dec_input, dec_state, enc_output)
        pred = dec_output.argmax(axis=1)
        pred_token = out_vocab.idx_to_token[int(pred.asscalar())]
        if pred_token == EOS:  # 当任一时间步搜索出EOS时，输出序列即完成
            break
        else:
            output_tokens.append(pred_token)
            dec_input = pred
    return output_tokens

简单测试一下模型。输入法语句子`ils regardent.`，翻译后的英语句子应该是`they are watching.`。

In [46]:
input_seq = 'ils regardent .'
translate(encoder, decoder, input_seq, max_seq_len)

['they', 'are', 'watching', '.']

### 10.12.5 评价翻译结果
评价机器翻译结果通常使用`BLEU`(Bilingual Evaluation Understudy)。对于模型预测序列中任意的子序列，`BLEU`考察这个子序列是否出现在标签序列中。

具体来说，设词数为$n$的子序列的精度为$p_n$。它是预测序列与标签序列匹配词数为$n$的子序列的数量与预测序列中词数为$n$的子序列的数量之比。举个例子，假设标签序列为$A$、$B$、$C$、$D$、$E$、$F$，预测序列为$A$、$B$、$B$、$C$、$D$，那么$p_1 = 4/5,\ p_2 = 3/4,\ p_3 = 1/3,\ p_4 = 0$。设$len_{\text{label}}$和$len_{\text{pred}}$分别为标签序列和预测序列的词数，那么，BLEU的定义为

$$ \exp\left(\min\left(0, 1 - \frac{len_{\text{label}}}{len_{\text{pred}}}\right)\right) \prod_{n=1}^k p_n^{1/2^n},$$

其中$k$是我们希望匹配的子序列的最大词数。可以看到当预测序列和标签序列完全一致时，BLEU为1。

因为匹配较长子序列比匹配较短子序列更难，BLEU对匹配较长子序列的精度赋予了更大权重。例如，当$p_n$固定在0.5时，随着$n$的增大，$0.5^{1/2} \approx 0.7, 0.5^{1/4} \approx 0.84, 0.5^{1/8} \approx 0.92, 0.5^{1/16} \approx 0.96$。另外，模型预测较短序列往往会得到较高$p_n$值。因此，上式中连乘项前面的系数是为了惩罚较短的输出而设的。举个例子，当$k=2$时，假设标签序列为$A$、$B$、$C$、$D$、$E$、$F$，而预测序列为$A$、$B$。虽然$p_1 = p_2 = 1$，但惩罚系数$\exp(1-6/2) \approx 0.14$，因此BLEU也接近0.14。

下面来实现BLEU的计算。

In [47]:
def bleu(pred_tokens, label_tokens, k):
    len_pred, len_label = len(pred_tokens), len(label_tokens)
    score = math.exp(min(0, 1 - len_label / len_pred))
    for n in range(1, k + 1):
        num_matches, label_subs = 0, collections.defaultdict(int)
        for i in range(len_label - n + 1):
            label_subs[''.join(label_tokens[i: i + n])] += 1
        for i in range(len_pred - n + 1):
            if label_subs[''.join(pred_tokens[i: i + n])] > 0:
                num_matches += 1
                label_subs[''.join(pred_tokens[i: i + n])] -= 1
        score *= math.pow(num_matches / (len_pred - n + 1), math.pow(0.5, n))
    return score

接下来，定义一个辅助打印函数。

In [48]:
def score(input_seq, label_seq, k):
    pred_tokens = translate(encoder, decoder, input_seq, max_seq_len)
    label_tokens = label_seq.split(' ')
    print('bleu %.3f, predict: %s' % (bleu(pred_tokens, label_tokens, k),
                                      ' '.join(pred_tokens)))

预测正确则分数为1。

In [49]:
score('ils regardent .', 'they are watching .', k=2)

bleu 1.000, predict: they are watching .


测试一个不在训练集中的样本。

In [50]:
score('ils sont canadiens .', 'they are canadian .', k=2)

bleu 0.658, predict: they are actors .
